troviamo massimi e minimi di lyso e adc

In [29]:
import pickle
import os
import numpy as np

Cerchiamo i minimi e i massimo sia per il lyso che per ADC
Il lyso è rappresentato dagli ultimi 9 numeri e ADC dai primi 31

In [30]:
with open('/home/private/Hepd/Dataset_4/massimi_minimi/massimo_adc_train.pkl','rb') as f:
    massimo_adc=pickle.load(f)
    massimo_adc_train=massimo_adc
with open('/home/private/Hepd/Dataset_4/massimi_minimi/massimo_adc_test.pkl','rb') as f:
    massimo_adc_test=pickle.load(f)
    if massimo_adc<massimo_adc_test:
        massimo_adc=massimo_adc_test
with open('/home/private/Hepd/Dataset_4/massimi_minimi/massimo_adc_validation.pkl','rb') as f:
    massimo_adc_validation=pickle.load(f)
    if massimo_adc<massimo_adc_validation:
        massimo_adc=massimo_adc_validation

In [33]:
with open('/home/private/Hepd/Dataset_4/massimi_minimi/minimo_adc_train.pkl','rb') as f:
    minimo_adc=pickle.load(f)
    minimo_adc_train = minimo_adc
with open('/home/private/Hepd/Dataset_4/massimi_minimi/minimo_adc_test.pkl','rb')as f:
    minimo_adc_test=pickle.load(f)
    if minimo_adc_test<minimo_adc:
        minimo_adc=minimo_adc_test
with open('/home/private/Hepd/Dataset_4/massimi_minimi/minimo_adc_validation.pkl','rb') as f:
    minimo_adc_validation=pickle.load(f)
    if minimo_adc_validation<minimo_adc:
        minimo_adc=minimo_adc_validation

In [34]:
with open('/home/private/Hepd/Dataset_4/massimi_minimi/massimo_lyso_train.pkl','rb') as f:
    massimo_lyso=pickle.load(f)
    massimo_lyso_train = massimo_lyso
with open('/home/private/Hepd/Dataset_4/massimi_minimi/massimo_lyso_test.pkl','rb') as f:
    massimo_lyso_test=pickle.load(f)
    if massimo_lyso_test>massimo_lyso:
        massimo_lyso=massimo_lyso_test
with open('/home/private/Hepd/Dataset_4/massimi_minimi/massimo_lyso_validation.pkl','rb')as f:
    massimo_lyso_validation=pickle.load(f)
    if massimo_lyso_validation>massimo_lyso:
        massimo_lyso=massimo_lyso_validation

In [35]:
with open('/home/private/Hepd/Dataset_4/massimi_minimi/minimo_lyso_train.pkl','rb') as f:
    minimo_lyso=pickle.load(f)
    minimo_lyso_train = minimo_lyso
with open('/home/private/Hepd/Dataset_4/massimi_minimi/minimo_lyso_test.pkl','rb') as f:
    minimo_lyso_test=pickle.load(f)
    if minimo_lyso_test<minimo_lyso:
        minimo_lyso=minimo_lyso_test
with open('/home/private/Hepd/Dataset_4/massimi_minimi/minimo_lyso_validation.pkl','rb') as f:
    minimo_lyso_validation=pickle.load(f)
    if minimo_lyso_validation<minimo_lyso:
        minimo_lyso=minimo_lyso_validation

In [36]:
train_labels_dir='/home/private/Hepd/Dataset_4/train/train_labels'
test_labels_dir='/home/private/Hepd/Dataset_4/test/test_labels'
validation_labels_dir='/home/private/Hepd/Dataset_4/validation/validation_labels'

num_batches_train=len([f for f in os.listdir(train_labels_dir) if os.path.isfile(os.path.join(train_labels_dir, f))])
num_batches_test=len([f for f in os.listdir(test_labels_dir) if os.path.isfile(os.path.join(test_labels_dir, f))])
num_batches_validation=len([f for f in os.listdir(validation_labels_dir) if os.path.isfile(os.path.join(validation_labels_dir, f))])

Normalizziamo sui utilizzando i massimi e i minimi del train_set non quelli generali di tutto il dataset
- moltiplichiamo il massimo del train per un fattore (> di 1) in modo da essere sicuri che il massimo di normalizzazione possa essere superiore a tutti i valori del dataset (scegliamo di aumetnare il massimo del 10%)
- moltiplichiamo il minimo del train per un fattore (< 1) in modo da essere sicuri che il minimo di normalizzazione possa essere inferiore a tutti i valori del dataset (scegliamo di dimunuire il minimo del 10%).

FACENDO QUESTA OPERAZIONE DOBBIAMO FARE ATTENZIONE SUL SEGNO DI QUESTI MASSIMI E QUESTI MINIMI, SE SONO NEGATIVI DOBBIAMO INVERTIRE I COEFFICENTI

In [37]:
increasing_coefficent = 1.1
decreasing_coefficent = 0.9
if massimo_adc_train > 0:
    massimo_adc_train_increased = massimo_adc_train * increasing_coefficent
else:
    massimo_adc_train_increased = massimo_adc_train * decreasing_coefficent

if massimo_lyso_train > 0:
    massimo_lyso_train_increased = massimo_lyso_train * increasing_coefficent
else:
    massimo_lyso_train_increased = massimo_lyso_train * decreasing_coefficent

if minimo_adc_train > 0:
    minimo_adc_train_decreased = minimo_adc_train * decreasing_coefficent
else:
    minimo_adc_train_decreased = minimo_adc_train * increasing_coefficent

if minimo_lyso_train > 0:
    minimo_lyso_train_decreased = minimo_lyso_train * decreasing_coefficent
else:
    minimo_lyso_train_decreased = minimo_lyso_train * increasing_coefficent

Normalizziamo i valori i dati in base al massimo increased e al minimo decreased

In [39]:
for i in range(num_batches_train):
    with open('/home/private/Hepd/Dataset_4/train/train_data/'+str(i)+'.pkl','rb') as f:
        data=pickle.load(f)
        if len(data)!=0:
            data_norm=np.zeros((len(data),len(data[0])))
        else: break
        for j in range(len(data)):
            norm_adc=((data[j][:31]-minimo_adc_train_decreased)/(massimo_adc_train_increased-minimo_adc_train_decreased))
            norm_lyso=((data[j][31:]-minimo_lyso_train_decreased)/(massimo_lyso_train_increased-minimo_lyso_train_decreased))
            data_norm[j]=np.concatenate((norm_adc,norm_lyso))
    with open('/home/private/Hepd/Dataset_4/train/train_norm_data/'+str(i)+'.pkl','wb') as f:
        pickle.dump(data_norm,f)


for i in range(num_batches_test):
    with open('/home/private/Hepd/Dataset_4/test/test_data/'+str(i)+'.pkl','rb') as f:
        data=pickle.load(f)
        if len(data)!=0:
            data_norm=np.zeros((len(data),len(data[0])))
        else: break
        for j in range(len(data)):
            norm_adc=((data[j][:31]-minimo_adc_train_decreased)/(massimo_adc_train_increased-minimo_adc_train_decreased))
            norm_lyso=((data[j][31:]-minimo_lyso_train_decreased)/(massimo_lyso_train_increased-minimo_lyso_train_decreased))
            data_norm[j]=np.concatenate((norm_adc,norm_lyso))
    with open('/home/private/Hepd/Dataset_4/test/test_norm_data/'+str(i)+'.pkl','wb') as f:
        pickle.dump(data_norm,f)


for i in range(num_batches_validation):
    with open('/home/private/Hepd/Dataset_4/validation/validation_data/'+str(i)+'.pkl','rb') as f:
        data=pickle.load(f)
        if len(data)!=0:
            data_norm=np.zeros((len(data),len(data[0])))
        else: break
        for j in range(len(data)):
            norm_adc=((data[j][:31]-minimo_adc_train_decreased)/(massimo_adc_train_increased-minimo_adc_train_decreased))
            norm_lyso=((data[j][31:]-minimo_lyso_train_decreased)/(massimo_lyso_train_increased-minimo_lyso_train_decreased))
            data_norm[j]=np.concatenate((norm_adc,norm_lyso))
    with open('/home/private/Hepd/Dataset_4/validation/validation_norm_data/'+str(i)+'.pkl','wb') as f:
        pickle.dump(data_norm,f)